In [1]:
import numpy as np

class CustomDecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        num_samples, num_features = X.shape
        unique_classes = np.unique(y)

        # Stopping conditions
        if len(unique_classes) == 1:
            return {'class': unique_classes[0]}
        if num_samples == 0 or (self.max_depth and depth >= self.max_depth):
            return {'class': np.bincount(y).argmax()}

        best_info_gain = -float('inf')
        best_split = None

        for feature_idx in range(num_features):
            thresholds = np.unique(X[:, feature_idx])
            for threshold in thresholds:
                left_mask = X[:, feature_idx] <= threshold
                right_mask = ~left_mask
                left_y = y[left_mask]
                right_y = y[right_mask]
                info_gain = self._information_gain(y, left_y, right_y)
                if info_gain > best_info_gain:
                    best_info_gain = info_gain
                    best_split = {'feature_idx': feature_idx, 'threshold': threshold,
                                  'left_y': left_y, 'right_y': right_y}

        if best_split is None:
            return {'class': np.bincount(y).argmax()}

        left_tree = self._build_tree(X[best_split['left_y']], best_split['left_y'], depth + 1)
        right_tree = self._build_tree(X[best_split['right_y']], best_split['right_y'], depth + 1)

        return {'feature_idx': best_split['feature_idx'], 'threshold': best_split['threshold'],
                'left_tree': left_tree, 'right_tree': right_tree}

    def _information_gain(self, parent, left, right):
        parent_entropy = self._entropy(parent)
        left_entropy = self._entropy(left)
        right_entropy = self._entropy(right)
        weighted_avg_entropy = (len(left) / len(parent)) * left_entropy + (len(right) / len(parent)) * right_entropy
        return parent_entropy - weighted_avg_entropy

    def _entropy(self, y):
        class_probs = np.bincount(y) / len(y)
        return -np.sum(class_probs * np.log2(class_probs + 1e-9))  # Added small epsilon

    def predict(self, X):
        return [self._predict_single(x, self.tree) for x in X]

    def _predict_single(self, x, tree):
        if 'class' in tree:
            return tree['class']
        feature_val = x[tree['feature_idx']]
        if feature_val <= tree['threshold']:
            return self._predict_single(x, tree['left_tree'])
        else:
            return self._predict_single(x, tree['right_tree'])


In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Load the Iris dataset
data = load_iris()
X = data.data
y = data.target

# Split into training and test sets (80% training, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
# Train the custom decision tree
custom_tree = CustomDecisionTree(max_depth=3)
custom_tree.fit(X_train, y_train)

# Predict on the test set
y_pred_custom = custom_tree.predict(X_test)

# Calculate accuracy
accuracy_custom = accuracy_score(y_test, y_pred_custom)
print(f"Custom Decision Tree Accuracy: {accuracy_custom:.4f}")


Custom Decision Tree Accuracy: 0.8000


In [4]:
# Train the Scikit-learn decision tree
sklearn_tree = DecisionTreeClassifier(max_depth=3, random_state=42)
sklearn_tree.fit(X_train, y_train)

# Predict on the test set
y_pred_sklearn = sklearn_tree.predict(X_test)

# Calculate accuracy
accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)
print(f"Scikit-learn Decision Tree Accuracy: {accuracy_sklearn:.4f}")


Scikit-learn Decision Tree Accuracy: 1.0000


In [5]:
print(f"Accuracy Comparison:")
print(f"Custom Decision Tree: {accuracy_custom:.4f}")
print(f"Scikit-learn Decision Tree: {accuracy_sklearn:.4f}")


Accuracy Comparison:
Custom Decision Tree: 0.8000
Scikit-learn Decision Tree: 1.0000


In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Load the Wine dataset
wine = load_wine()
X_wine = wine.data
y_wine = wine.target

# Split into training and test sets (80% training, 20% test)
X_train_wine, X_test_wine, y_train_wine, y_test_wine = train_test_split(X_wine, y_wine, test_size=0.2, random_state=42)

# Train Decision Tree Classifier
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(X_train_wine, y_train_wine)
y_pred_dt = dt_classifier.predict(X_test_wine)
f1_dt = f1_score(y_test_wine, y_pred_dt, average='weighted')
print(f"Decision Tree F1 Score: {f1_dt:.4f}")

# Train Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train_wine, y_train_wine)
y_pred_rf = rf_classifier.predict(X_test_wine)
f1_rf = f1_score(y_test_wine, y_pred_rf, average='weighted')
print(f"Random Forest F1 Score: {f1_rf:.4f}")


Decision Tree F1 Score: 0.9440
Random Forest F1 Score: 1.0000


In [7]:
from sklearn.model_selection import GridSearchCV

# Hyperparameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5]
}

# Create a Random Forest model
rf = RandomForestClassifier(random_state=42)

# GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', verbose=2)
grid_search.fit(X_train_wine, y_train_wine)

# Best parameters and score
print(f"Best Hyperparameters: {grid_search.best_params_}")
print(f"Best Cross-validation Accuracy: {grid_search.best_score_:.4f}")


Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END .max_depth=5, min_samples_split=2, n_estimators=100; total time=   0.4s
[CV] END .max_depth=5, min_samples_split=2, n_estimators=100; total time=   0.3s
[CV] END .max_depth=5, min_samples_split=2, n_estimators=100; total time=   0.4s
[CV] END .max_depth=5, min_samples_split=2, n_estimators=100; total time=   0.3s
[CV] END .max_depth=5, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END .max_depth=5, min_samples_split=2, n_estimators=200; total time=   0.3s
[CV] END .max_depth=5, min_samples_split=2, n_estimators=200; total time=   0.3s
[CV] END .max_depth=5, min_samples_split=2, n_estimators=200; total time=   0.4s
[CV] END .max_depth=5, min_samples_split=2, n_estimators=200; total time=   0.5s
[CV] END .max_depth=5, min_samples_split=2, n_estimators=200; total time=   0.6s
[CV] END .max_depth=5, min_samples_split=5, n_estimators=100; total time=   0.3s
[CV] END .max_depth=5, min_samples_split=5, n_es

In [8]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error

# Load Wine dataset for regression (use a continuous target variable)
# For this, we'll consider "target" as the regression target
X_wine_reg = wine.data
y_wine_reg = wine.target

# Split into training and test sets (80% training, 20% test)
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_wine_reg, y_wine_reg, test_size=0.2, random_state=42)

# Train Decision Tree Regressor
dt_regressor = DecisionTreeRegressor(random_state=42)
dt_regressor.fit(X_train_reg, y_train_reg)
y_pred_dt_reg = dt_regressor.predict(X_test_reg)
mse_dt = mean_squared_error(y_test_reg, y_pred_dt_reg)
print(f"Decision Tree Regressor MSE: {mse_dt:.4f}")

# Train Random Forest Regressor
rf_regressor = RandomForestRegressor(random_state=42)
rf_regressor.fit(X_train_reg, y_train_reg)
y_pred_rf_reg = rf_regressor.predict(X_test_reg)
mse_rf = mean_squared_error(y_test_reg, y_pred_rf_reg)
print(f"Random Forest Regressor MSE: {mse_rf:.4f}")

# Hyperparameter tuning using RandomizedSearchCV
param_grid_reg = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5]
}

rf_reg = RandomForestRegressor(random_state=42)

random_search = RandomizedSearchCV(estimator=rf_reg, param_distributions=param_grid_reg, n_iter=10, cv=5, verbose=2)
random_search.fit(X_train_reg, y_train_reg)

# Best parameters and score
print(f"Best Hyperparameters: {random_search.best_params_}")
print(f"Best Cross-validation MSE: {random_search.best_score_:.4f}")


Decision Tree Regressor MSE: 0.1667
Random Forest Regressor MSE: 0.0648
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END .max_depth=5, min_samples_split=5, n_estimators=100; total time=   0.6s
[CV] END .max_depth=5, min_samples_split=5, n_estimators=100; total time=   0.4s
[CV] END .max_depth=5, min_samples_split=5, n_estimators=100; total time=   0.8s
[CV] END .max_depth=5, min_samples_split=5, n_estimators=100; total time=   0.4s
[CV] END .max_depth=5, min_samples_split=5, n_estimators=100; total time=   0.2s
[CV] END .max_depth=5, min_samples_split=5, n_estimators=200; total time=   0.4s
[CV] END .max_depth=5, min_samples_split=5, n_estimators=200; total time=   0.3s
[CV] END .max_depth=5, min_samples_split=5, n_estimators=200; total time=   0.3s
[CV] END .max_depth=5, min_samples_split=5, n_estimators=200; total time=   0.3s
[CV] END .max_depth=5, min_samples_split=5, n_estimators=200; total time=   0.3s
[CV] END max_depth=None, min_samples_split=5, n_estimator